In [5]:
import flwr as fl
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms 
from torchvision.datasets import CIFAR10, FashionMNIST, SVHN
 
# Constants
BATCH_SIZE = 64
LEARNING_RATE = 0.001
EPOCHS = 5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
# Define the model with dataset-specific layers
class CNN_Model(nn.Module):
    def __init__(self, dataset_name):
        super(CNN_Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.AvgPool2d(2, 2)
        self.fc_config = nn.Linear(64 * 8 * 8, 128)
 
        # Dataset-specific layers
        if dataset_name == "cifar10":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 10)
            )
        elif dataset_name == "fashion_mnist":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 32),
                nn.ReLU(),
                nn.Linear(32, 10)
            )
        elif dataset_name == "svhn":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 10)
            )
 
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc_config(x))
        x = self.fc_op(x)
        return F.softmax(x, dim=1)
 
# Define the Flower client
class FLClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, test_loader, client_id):
        self.model = model.to(DEVICE)
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=LEARNING_RATE)
        self.client_id = client_id
 
    def get_parameters(self, config=None):
        params = {
            k: v.cpu().numpy()
            for k, v in self.model.state_dict().items()
            if not k.startswith("fc_op")
        }
        return list(params.values())
 
    def set_parameters(self, parameters):
        state_dict = self.model.state_dict()
        shared_keys = [k for k in state_dict.keys() if not k.startswith("fc_op")]
        for key, param in zip(shared_keys, parameters):
            state_dict[key] = torch.tensor(param)
        self.model.load_state_dict(state_dict, strict=False)
 
    def fine_tune(self):
        self.model.train()
        for epoch in range(2):  # Fine-tune for 2 local epochs
            for images, labels in self.train_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
 
    def fit(self, parameters, config):
        self.set_parameters(parameters)  # Set global model parameters
        self.model.train()
        for epoch in range(EPOCHS):  # Train on local data
            for images, labels in self.train_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
        # Perform fine-tuning
        self.fine_tune()
 
        return self.get_parameters(config), len(self.train_loader.dataset), {}
 
    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        correct, total = 0, 0
        loss = 0.0
        with torch.no_grad():
            for images, labels in self.test_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = self.model(images)
                loss += self.criterion(outputs, labels).item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
        print(f"Client {self.client_id}: Accuracy = {accuracy}")
        return float(loss) / len(self.test_loader), len(self.test_loader.dataset), {"accuracy": accuracy}
 
# Load the datasets
def load_data(dataset_name):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    if dataset_name == "cifar10":
        train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
        test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
    elif dataset_name == "fashion_mnist":
        transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,)),
        ])
        train_dataset = FashionMNIST(root='./data', train=True, download=True, transform=transform)
        test_dataset = FashionMNIST(root='./data', train=False, download=True, transform=transform)
    elif dataset_name == "svhn":
        train_dataset = SVHN(root='./data', split='train', download=True, transform=transform)
        test_dataset = SVHN(root='./data', split='test', download=True, transform=transform)
    else:
        raise ValueError(f"Unknown dataset: {dataset_name}")
 
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    return train_loader, test_loader
 
# Updated client_fn to avoid using Context's client_id
def client_fn(cid: str) -> fl.client.Client:
    if cid == "0":
        return cifar_client
    elif cid == "1":
        return fashion_client
    elif cid == "2":
        return svhn_client
 
# Start federated learning
def start_federated_learning():
    global cifar_client, fashion_client, svhn_client
    cifar_train_loader, cifar_test_loader = load_data("cifar10")
    fashion_train_loader, fashion_test_loader = load_data("fashion_mnist")
    svhn_train_loader, svhn_test_loader = load_data("svhn")
 
    cifar_client = FLClient(CNN_Model("cifar10"), cifar_train_loader, cifar_test_loader, "CIFAR-10")
    fashion_client = FLClient(CNN_Model("fashion_mnist"), fashion_train_loader, fashion_test_loader, "Fashion-MNIST")
    svhn_client = FLClient(CNN_Model("svhn"), svhn_train_loader, svhn_test_loader, "SVHN")
 
    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=3,
        client_resources={"num_cpus": 2, "num_gpus": 1},
        config=fl.server.ServerConfig(num_rounds=10),  # Increased number of rounds
        strategy=fl.server.strategy.FedAvg(
            fraction_fit=1.0,
            min_fit_clients=3,
            min_available_clients=3,
        )
    )
 
if __name__ == "__main__":
    start_federated_learning()

Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout
2024-12-04 17:56:55,005	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'object_store_memory': 6006627532.0, 'memory': 12013255067.0, 'CPU': 12.0, 'node:131.227.65.97': 1.0, 'GPU': 1.0, 'accelerator_type:P4000': 1.0, 'node:__internal_head__': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 2, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(ClientAppActor pid=2043586) /user/HS402/as05511/.local/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module impl

(ClientAppActor pid=2043586) Client CIFAR-10: Accuracy = 0.1807


(ClientAppActor pid=2043586) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=2043586) 
(ClientAppActor pid=2043586)             This is a deprecated feature. It will be removed
(ClientAppActor pid=2043586)             entirely in future versions of Flower.
(ClientAppActor pid=2043586)         
(ClientAppActor pid=2043586) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=2043586) Client SVHN: Accuracy = 0.193799938537185


...

(ClientAppActor pid=2043586) Client Fashion-MNIST: Accuracy = 0.7636


(ClientAppActor pid=2043586) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=2043586) 
(ClientAppActor pid=2043586)             This is a deprecated feature. It will be removed
(ClientAppActor pid=2043586)             entirely in future versions of Flower.
(ClientAppActor pid=2043586)         
(ClientAppActor pid=2043586) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=2043586) Client SVHN: Accuracy = 0.4001613398893669


(ClientAppActor pid=2043586) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=2043586) 
(ClientAppActor pid=2043586)             This is a deprecated feature. It will be removed
(ClientAppActor pid=2043586)             entirely in future versions of Flower.
(ClientAppActor pid=2043586)         
(ClientAppActor pid=2043586) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 round(s) in 3533.15s
INFO :      	History (loss, distributed):
INFO :      		round 1: 2.2923804188299663
INFO :  

(ClientAppActor pid=2043586) Client CIFAR-10: Accuracy = 0.2715


In [8]:
import flwr as fl
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms 
from torchvision.datasets import CIFAR10, FashionMNIST, SVHN
 
# Constants
BATCH_SIZE = 64
LEARNING_RATE = 0.001
EPOCHS = 5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
# Define the model with dataset-specific layers
class CNN_Model(nn.Module):
    def __init__(self, dataset_name):
        super(CNN_Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.AvgPool2d(2, 2)
        self.fc_config = nn.Linear(64 * 8 * 8, 128)
 
        # Dataset-specific layers
        if dataset_name == "cifar10":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 10)
            )
        elif dataset_name == "fashion_mnist":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 32),
                nn.ReLU(),
                nn.Linear(32, 10)
            )
        elif dataset_name == "svhn":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 10)
            )
 
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc_config(x))
        x = self.fc_op(x)
        return F.softmax(x, dim=1)
 
# Define the Flower client
class FLClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, test_loader, client_id):
        self.model = model.to(DEVICE)
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=LEARNING_RATE)
        self.client_id = client_id
 
    def get_parameters(self, config=None):
        params = {
            k: v.cpu().numpy()
            for k, v in self.model.state_dict().items()
            if not k.startswith("fc_op")
        }
        return list(params.values())
 
    def set_parameters(self, parameters):
        state_dict = self.model.state_dict()
        shared_keys = [k for k in state_dict.keys() if not k.startswith("fc_op")]
        for key, param in zip(shared_keys, parameters):
            state_dict[key] = torch.tensor(param)
        self.model.load_state_dict(state_dict, strict=False)
 
    def fine_tune(self):
        self.model.train()
        for epoch in range(2):  # Fine-tune for 2 local epochs
            for images, labels in self.train_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
 
    def fit(self, parameters, config):
        self.set_parameters(parameters)  # Set global model parameters
        self.model.train()
        for epoch in range(EPOCHS):  # Train on local data
            for images, labels in self.train_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
        # Perform fine-tuning
        self.fine_tune()
 
        return self.get_parameters(config), len(self.train_loader.dataset), {}
 
    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        correct, total = 0, 0
        loss = 0.0
        with torch.no_grad():
            for images, labels in self.test_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = self.model(images)
                loss += self.criterion(outputs, labels).item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
        print(f"Client {self.client_id}: Accuracy = {accuracy}")
        return float(loss) / len(self.test_loader), len(self.test_loader.dataset), {"accuracy": accuracy}
 
# Load the datasets
def load_data(dataset_name):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    if dataset_name == "cifar10":
        train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
        test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
    elif dataset_name == "fashion_mnist":
        transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,)),
        ])
        train_dataset = FashionMNIST(root='./data', train=True, download=True, transform=transform)
        test_dataset = FashionMNIST(root='./data', train=False, download=True, transform=transform)
    elif dataset_name == "svhn":
        train_dataset = SVHN(root='./data', split='train', download=True, transform=transform)
        test_dataset = SVHN(root='./data', split='test', download=True, transform=transform)
    else:
        raise ValueError(f"Unknown dataset: {dataset_name}")
 
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    return train_loader, test_loader
 
# Updated client_fn to avoid using Context's client_id
def client_fn(cid: str) -> fl.client.Client:
    if cid == "0":
        return cifar_client
    elif cid == "1":
        return fashion_client
    elif cid == "2":
        return svhn_client
 
# Start federated learning
def start_federated_learning():
    global cifar_client, fashion_client, svhn_client
    cifar_train_loader, cifar_test_loader = load_data("cifar10")
    fashion_train_loader, fashion_test_loader = load_data("fashion_mnist")
    svhn_train_loader, svhn_test_loader = load_data("svhn")
 
    cifar_client = FLClient(CNN_Model("cifar10"), cifar_train_loader, cifar_test_loader, "CIFAR-10")
    fashion_client = FLClient(CNN_Model("fashion_mnist"), fashion_train_loader, fashion_test_loader, "Fashion-MNIST")
    svhn_client = FLClient(CNN_Model("svhn"), svhn_train_loader, svhn_test_loader, "SVHN")
 
    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=3,
        client_resources={"num_cpus": 2, "num_gpus": 1},
        config=fl.server.ServerConfig(num_rounds=5),  # Increased number of rounds
        strategy=fl.server.strategy.FedAvg(
            fraction_fit=1.0,
            min_fit_clients=3,
            min_available_clients=3,
        )
    )
 
if __name__ == "__main__":
    start_federated_learning()

Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-12-11 19:18:09,185	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:P4000': 1.0, 'node:__internal_head__': 1.0, 'memory': 9905858151.0, 'node:131.227.65.97': 1.0, 'CPU': 12.0, 'GPU': 1.0, 'object_store_memory': 4952929075.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 2, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(ClientAppActor pid=3499243) /user/HS402/as05511/.local/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implic

(ClientAppActor pid=3499243) Client CIFAR-10: Accuracy = 0.1501


(ClientAppActor pid=3499243) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3499243) 
(ClientAppActor pid=3499243)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3499243)             entirely in future versions of Flower.
(ClientAppActor pid=3499243)         
(ClientAppActor pid=3499243) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=3499243) Client Fashion-MNIST: Accuracy = 0.4043


...

(ClientAppActor pid=3499243) Client Fashion-MNIST: Accuracy = 0.7145


(ClientAppActor pid=3499243) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3499243) 
(ClientAppActor pid=3499243)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3499243)             entirely in future versions of Flower.
(ClientAppActor pid=3499243)         
(ClientAppActor pid=3499243) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=3499243) Client SVHN: Accuracy = 0.4594345421020283


(ClientAppActor pid=3499243) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3499243) 
(ClientAppActor pid=3499243)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3499243)             entirely in future versions of Flower.
(ClientAppActor pid=3499243)         
(ClientAppActor pid=3499243) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 1631.11s
INFO :      	History (loss, distributed):
INFO :      		round 1: 2.2919704759266084
INFO :   

(ClientAppActor pid=3499243) Client CIFAR-10: Accuracy = 0.247


In [8]:
import flwr as fl
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10, FashionMNIST, SVHN

# Constants
BATCH_SIZE = 128
LEARNING_RATE = 0.001
EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model with dataset-specific layers
class CNN_Model(nn.Module):
    def __init__(self, dataset_name):
        super(CNN_Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.AvgPool2d(2, 2)
        self.fc_config = nn.Linear(64 * 8 * 8, 128)

        # Dataset-specific layers
        if dataset_name == "cifar10":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 10)
            )
        elif dataset_name == "fashion_mnist":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 32),
                nn.ReLU(),
                nn.Linear(32, 10)
            )
        elif dataset_name == "svhn":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 10)
            )

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc_config(x))
        x = self.fc_op(x)
        return F.softmax(x, dim=1)

# Define the Flower client with FedProx
class FLClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, test_loader, client_id):
        self.model = model.to(DEVICE)
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=LEARNING_RATE)
        self.client_id = client_id

    def get_parameters(self, config=None):
        params = {
            k: v.cpu().numpy()
            for k, v in self.model.state_dict().items()
            if not k.startswith("fc_op")
        }
        return list(params.values())

    def set_parameters(self, parameters):
        state_dict = self.model.state_dict()
        shared_keys = [k for k in state_dict.keys() if not k.startswith("fc_op")]
        for key, param in zip(shared_keys, parameters):
            state_dict[key] = torch.tensor(param)
        self.model.load_state_dict(state_dict, strict=False)

    def fit(self, parameters, config):
        # Set global parameters on the local model
        self.set_parameters(parameters)
        self.model.train()
        
        # Save global model parameters for proximity
        global_params = [param.clone().detach().to(DEVICE) for param in self.model.parameters()]
        
        for epoch in range(EPOCHS):
            for images, labels in self.train_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                
                # Add proximal term to the loss
                prox_term = 0.0
                for param, global_param in zip(self.model.parameters(), global_params):
                    prox_term += torch.norm(param - global_param) ** 2
                loss += (0.01 / 2) * prox_term  # 0.01 is the proximal coefficient
                
                loss.backward()
                self.optimizer.step()
        
        return self.get_parameters(config), len(self.train_loader.dataset), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        correct, total = 0, 0
        loss = 0.0
        with torch.no_grad():
            for images, labels in self.test_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = self.model(images)
                loss += self.criterion(outputs, labels).item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
        print(f"Client {self.client_id}: Accuracy = {accuracy}")
        return float(loss) / len(self.test_loader), len(self.test_loader.dataset), {"accuracy": accuracy}

# Load the datasets
def load_data(dataset_name):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    if dataset_name == "cifar10":
        train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
        test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
    elif dataset_name == "fashion_mnist": 
        transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,)),
        ])
        train_dataset = FashionMNIST(root='./data', train=True, download=True, transform=transform)
        test_dataset = FashionMNIST(root='./data', train=False, download=True, transform=transform)
    elif dataset_name == "svhn":
        train_dataset = SVHN(root='./data', split='train', download=True, transform=transform)
        test_dataset = SVHN(root='./data', split='test', download=True, transform=transform)
    else:
        raise ValueError(f"Unknown dataset: {dataset_name}")

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    return train_loader, test_loader

# Updated client_fn to avoid using Context's client_id
def client_fn(cid: str) -> fl.client.Client:
    if cid == "0":
        return cifar_client
    elif cid == "1":
        return fashion_client
    elif cid == "2":
        return svhn_client

# Start federated learning
def start_federated_learning():
    global cifar_client, fashion_client, svhn_client
    cifar_train_loader, cifar_test_loader = load_data("cifar10")
    fashion_train_loader, fashion_test_loader = load_data("fashion_mnist")
    svhn_train_loader, svhn_test_loader = load_data("svhn")

    cifar_client = FLClient(CNN_Model("cifar10"), cifar_train_loader, cifar_test_loader, "CIFAR-10")
    fashion_client = FLClient(CNN_Model("fashion_mnist"), fashion_train_loader, fashion_test_loader, "Fashion-MNIST")
    svhn_client = FLClient(CNN_Model("svhn"), svhn_train_loader, svhn_test_loader, "SVHN")

    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=3,
        client_resources={"num_cpus": 4, "num_gpus": 1},
        config=fl.server.ServerConfig(num_rounds=20),  # Increased number of rounds
        strategy=fl.server.strategy.FedAvg(
            fraction_fit=1.0,
            min_fit_clients=3,
            min_available_clients=3,
        )
    )

if __name__ == "__main__":
    start_federated_learning()


Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


INFO :      Starting Flower simulation, config: num_rounds=20, no round_timeout
2024-12-04 19:38:04,941	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'memory': 11841161627.0, 'object_store_memory': 5920580812.0, 'GPU': 1.0, 'CPU': 12.0, 'node:131.227.65.97': 1.0, 'node:__internal_head__': 1.0, 'accelerator_type:P4000': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 4, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(ClientAppActor pid=2062115) /user/HS402/as05511/.local/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module impl

(ClientAppActor pid=2062115) Client Fashion-MNIST: Accuracy = 0.2716


(ClientAppActor pid=2062115) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=2062115) 
(ClientAppActor pid=2062115)             This is a deprecated feature. It will be removed
(ClientAppActor pid=2062115)             entirely in future versions of Flower.
(ClientAppActor pid=2062115)         
(ClientAppActor pid=2062115) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=2062115) Client CIFAR-10: Accuracy = 0.134


...

(ClientAppActor pid=2062115) Client Fashion-MNIST: Accuracy = 0.4806


(ClientAppActor pid=2062115) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=2062115) 
(ClientAppActor pid=2062115)             This is a deprecated feature. It will be removed
(ClientAppActor pid=2062115)             entirely in future versions of Flower.
(ClientAppActor pid=2062115)         
(ClientAppActor pid=2062115) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=2062115) Client CIFAR-10: Accuracy = 0.2528


(ClientAppActor pid=2062115) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=2062115) 
(ClientAppActor pid=2062115)             This is a deprecated feature. It will be removed
(ClientAppActor pid=2062115)             entirely in future versions of Flower.
(ClientAppActor pid=2062115)         
(ClientAppActor pid=2062115) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 20 round(s) in 9588.11s
INFO :      	History (loss, distributed):
INFO :      		round 1: 2.292483091673799
INFO :   

(ClientAppActor pid=2062115) Client SVHN: Accuracy = 0.4267440073755378


In [1]:
#####SWATSSS

In [2]:
import flwr as fl

import torch

import torch.nn as nn

import torch.optim as optim

import torch.nn.functional as F

from torch.utils.data import DataLoader

import torchvision.transforms as transforms 

from torchvision.datasets import CIFAR10, FashionMNIST, SVHN
 
# Constants

BATCH_SIZE = 64

LEARNING_RATE = 0.001

EPOCHS = 5

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SWITCH_EPOCH = 3  # The epoch at which we switch from Adam to SGD
 
# Define the model with dataset-specific layers

class CNN_Model(nn.Module):

    def __init__(self, dataset_name):

        super(CNN_Model, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)

        self.pool = nn.AvgPool2d(2, 2)

        self.fc_config = nn.Linear(64 * 8 * 8, 128)
 
        # Dataset-specific layers

        if dataset_name == "cifar10":

            self.fc_op = nn.Sequential(

                nn.Linear(128, 64),

                nn.ReLU(),

                nn.Linear(64, 10)

            )

        elif dataset_name == "fashion_mnist":

            self.fc_op = nn.Sequential(

                nn.Linear(128, 32),

                nn.ReLU(),

                nn.Linear(32, 10)

            )

        elif dataset_name == "svhn":

            self.fc_op = nn.Sequential(

                nn.Linear(128, 64),

                nn.ReLU(),

                nn.Linear(64, 10)

            )
 
    def forward(self, x):

        x = F.relu(self.conv1(x))

        x = self.pool(x)

        x = F.relu(self.conv2(x))

        x = self.pool(x)

        x = x.view(-1, 64 * 8 * 8)

        x = F.relu(self.fc_config(x))

        x = self.fc_op(x)

        return F.softmax(x, dim=1)
 
# Define the Flower client

class FLClient(fl.client.NumPyClient):

    def __init__(self, model, train_loader, test_loader, client_id):

        self.model = model.to(DEVICE)

        self.train_loader = train_loader

        self.test_loader = test_loader

        self.criterion = nn.CrossEntropyLoss()

        self.optimizer = optim.Adam(self.model.parameters(), lr=LEARNING_RATE)

        self.sgd = None

        self.switched = False

        self.client_id = client_id
 
    def switch_to_sgd(self):

        # Switch to SGD optimizer

        self.sgd = optim.SGD(self.model.parameters(), lr=self.optimizer.param_groups[0]['lr'])

        self.switched = True

        print(f"Client {self.client_id}: Switched to SGD")
 
    def step_optimizer(self, epoch):

        # Switch from Adam to SGD if the conditions are met

        if not self.switched and epoch >= SWITCH_EPOCH:

            self.switch_to_sgd()

        return self.sgd if self.switched else self.optimizer
 
    def get_parameters(self, config=None):

        params = {

            k: v.cpu().numpy()

            for k, v in self.model.state_dict().items()

            if not k.startswith("fc_op")

        }

        return list(params.values())
 
    def set_parameters(self, parameters):

        state_dict = self.model.state_dict()

        shared_keys = [k for k in state_dict.keys() if not k.startswith("fc_op")]

        for key, param in zip(shared_keys, parameters):

            state_dict[key] = torch.tensor(param)

        self.model.load_state_dict(state_dict, strict=False)
 
    def fine_tune(self):

        self.model.train()

        for epoch in range(2):  # Fine-tune for 2 local epochs

            optimizer = self.step_optimizer(epoch)

            for images, labels in self.train_loader:

                images, labels = images.to(DEVICE), labels.to(DEVICE)

                optimizer.zero_grad()

                outputs = self.model(images)

                loss = self.criterion(outputs, labels)

                loss.backward()

                optimizer.step()
 
    def fit(self, parameters, config):

        self.set_parameters(parameters)  # Set global model parameters

        self.model.train()

        for epoch in range(EPOCHS):  # Train on local data

            optimizer = self.step_optimizer(epoch)

            for images, labels in self.train_loader:

                images, labels = images.to(DEVICE), labels.to(DEVICE)

                optimizer.zero_grad()

                outputs = self.model(images)

                loss = self.criterion(outputs, labels)

                loss.backward()

                optimizer.step()

        # Perform fine-tuning

        self.fine_tune()
 
        return self.get_parameters(config), len(self.train_loader.dataset), {}
 
    def evaluate(self, parameters, config):

        self.set_parameters(parameters)

        self.model.eval()

        correct, total = 0, 0

        loss = 0.0

        with torch.no_grad():

            for images, labels in self.test_loader:

                images, labels = images.to(DEVICE), labels.to(DEVICE)

                outputs = self.model(images)

                loss += self.criterion(outputs, labels).item()

                _, predicted = torch.max(outputs.data, 1)

                total += labels.size(0)

                correct += (predicted == labels).sum().item()

        accuracy = correct / total

        print(f"Client {self.client_id}: Accuracy = {accuracy}")

        return float(loss) / len(self.test_loader), len(self.test_loader.dataset), {"accuracy": accuracy}
 
# Load the datasets

def load_data(dataset_name):

    transform = transforms.Compose([

        transforms.Resize((32, 32)),

        transforms.ToTensor(),

        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),

    ])

    if dataset_name == "cifar10":

        train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)

        test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

    elif dataset_name == "fashion_mnist":

        transform = transforms.Compose([

            transforms.Grayscale(num_output_channels=3),

            transforms.Resize((32, 32)),

            transforms.ToTensor(),

            transforms.Normalize((0.5,), (0.5,)),

        ])

        train_dataset = FashionMNIST(root='./data', train=True, download=True, transform=transform)

        test_dataset = FashionMNIST(root='./data', train=False, download=True, transform=transform)

    elif dataset_name == "svhn":

        train_dataset = SVHN(root='./data', split='train', download=True, transform=transform)

        test_dataset = SVHN(root='./data', split='test', download=True, transform=transform)

    else:

        raise ValueError(f"Unknown dataset: {dataset_name}")
 
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    return train_loader, test_loader
 
# Updated client_fn to avoid using Context's client_id

def client_fn(cid: str) -> fl.client.Client:

    if cid == "0":

        return cifar_client

    elif cid == "1":

        return fashion_client

    elif cid == "2":

        return svhn_client
 
# Start federated learning

def start_federated_learning():

    global cifar_client, fashion_client, svhn_client

    cifar_train_loader, cifar_test_loader = load_data("cifar10")

    fashion_train_loader, fashion_test_loader = load_data("fashion_mnist")

    svhn_train_loader, svhn_test_loader = load_data("svhn")
 
    cifar_client = FLClient(CNN_Model("cifar10"), cifar_train_loader, cifar_test_loader, "CIFAR-10")

    fashion_client = FLClient(CNN_Model("fashion_mnist"), fashion_train_loader, fashion_test_loader, "Fashion-MNIST")

    svhn_client = FLClient(CNN_Model("svhn"), svhn_train_loader, svhn_test_loader, "SVHN")
 
    fl.simulation.start_simulation(

        client_fn=client_fn,

        num_clients=3,

        client_resources={"num_cpus": 2, "num_gpus": 1},

        config=fl.server.ServerConfig(num_rounds=10),  # Increased number of rounds

        strategy=fl.server.strategy.FedAvg(

            fraction_fit=1.0,

            min_fit_clients=3,

            min_available_clients=3,

        )

    )
 
if __name__ == "__main__":

    start_federated_learning()

 

2024-12-09 18:51:43,831	INFO util.py:154 -- Outdated packages:
  ipywidgets==6.0.0 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout
2024-12-09 18:53:15,213	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'GPU': 1.0, 'object_store_memory': 5514640588.0, 'memory': 11029281179.0, 'accelerator_type:P4000': 1.0, 'node:__internal_head__': 1.0, 'CPU': 12.0, 'node:131.227.65.97': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 2, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(ClientAppActor pid=3050191) /user/HS402/as05511/.local/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module impl

(ClientAppActor pid=3050191) Client SVHN: Switched to SGD


(ClientAppActor pid=3050191) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3050191) 
(ClientAppActor pid=3050191)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3050191)             entirely in future versions of Flower.
(ClientAppActor pid=3050191)         
(ClientAppActor pid=3050191) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


...

(ClientAppActor pid=3050191) Client Fashion-MNIST: Accuracy = 0.7722


(ClientAppActor pid=3050191) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3050191) 
(ClientAppActor pid=3050191)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3050191)             entirely in future versions of Flower.
(ClientAppActor pid=3050191)         
(ClientAppActor pid=3050191) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=3050191) Client CIFAR-10: Accuracy = 0.274


(ClientAppActor pid=3050191) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3050191) 
(ClientAppActor pid=3050191)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3050191)             entirely in future versions of Flower.
(ClientAppActor pid=3050191)         
(ClientAppActor pid=3050191) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 round(s) in 3284.93s
INFO :      	History (loss, distributed):
INFO :      		round 1: 2.2808811926651433
INFO :  

(ClientAppActor pid=3050191) Client SVHN: Accuracy = 0.4969268592501537


In [9]:
import flwr as fl

import torch

import torch.nn as nn

import torch.optim as optim

import torch.nn.functional as F

from torch.utils.data import DataLoader

import torchvision.transforms as transforms 

from torchvision.datasets import CIFAR10, FashionMNIST, SVHN
 
# Constants

BATCH_SIZE = 64

LEARNING_RATE = 0.001

EPOCHS = 5

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SWITCH_EPOCH = 3  # The epoch at which we switch from Adam to SGD
 
# Define the model with dataset-specific layers

class CNN_Model(nn.Module):

    def __init__(self, dataset_name):

        super(CNN_Model, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)

        self.pool = nn.AvgPool2d(2, 2)

        self.fc_config = nn.Linear(64 * 8 * 8, 128)
 
        # Dataset-specific layers

        if dataset_name == "cifar10":

            self.fc_op = nn.Sequential(

                nn.Linear(128, 64),

                nn.ReLU(),

                nn.Linear(64, 10)

            )

        elif dataset_name == "fashion_mnist":

            self.fc_op = nn.Sequential(

                nn.Linear(128, 32),

                nn.ReLU(),

                nn.Linear(32, 10)

            )

        elif dataset_name == "svhn":

            self.fc_op = nn.Sequential(

                nn.Linear(128, 64),

                nn.ReLU(),

                nn.Linear(64, 10)

            )
 
    def forward(self, x):

        x = F.relu(self.conv1(x))

        x = self.pool(x)

        x = F.relu(self.conv2(x))

        x = self.pool(x)

        x = x.view(-1, 64 * 8 * 8)

        x = F.relu(self.fc_config(x))

        x = self.fc_op(x)

        return F.softmax(x, dim=1)
 
# Define the Flower client

class FLClient(fl.client.NumPyClient):

    def __init__(self, model, train_loader, test_loader, client_id):

        self.model = model.to(DEVICE)

        self.train_loader = train_loader

        self.test_loader = test_loader

        self.criterion = nn.CrossEntropyLoss()

        self.optimizer = optim.Adam(self.model.parameters(), lr=LEARNING_RATE)

        self.sgd = None

        self.switched = False

        self.client_id = client_id
 
    def switch_to_sgd(self):

        # Switch to SGD optimizer

        self.sgd = optim.SGD(self.model.parameters(), lr=self.optimizer.param_groups[0]['lr'])

        self.switched = True

        print(f"Client {self.client_id}: Switched to SGD")
 
    def step_optimizer(self, epoch):

        # Switch from Adam to SGD if the conditions are met

        if not self.switched and epoch >= SWITCH_EPOCH:

            self.switch_to_sgd()

        return self.sgd if self.switched else self.optimizer
 
    def get_parameters(self, config=None):

        params = {

            k: v.cpu().numpy()

            for k, v in self.model.state_dict().items()

            if not k.startswith("fc_op")

        }

        return list(params.values())
 
    def set_parameters(self, parameters):

        state_dict = self.model.state_dict()

        shared_keys = [k for k in state_dict.keys() if not k.startswith("fc_op")]

        for key, param in zip(shared_keys, parameters):

            state_dict[key] = torch.tensor(param)

        self.model.load_state_dict(state_dict, strict=False)
 
    def fine_tune(self):

        self.model.train()

        for epoch in range(2):  # Fine-tune for 2 local epochs

            optimizer = self.step_optimizer(epoch)

            for images, labels in self.train_loader:

                images, labels = images.to(DEVICE), labels.to(DEVICE)

                optimizer.zero_grad()

                outputs = self.model(images)

                loss = self.criterion(outputs, labels)

                loss.backward()

                optimizer.step()
 
    def fit(self, parameters, config):

        self.set_parameters(parameters)  # Set global model parameters

        self.model.train()

        for epoch in range(EPOCHS):  # Train on local data

            optimizer = self.step_optimizer(epoch)

            for images, labels in self.train_loader:

                images, labels = images.to(DEVICE), labels.to(DEVICE)

                optimizer.zero_grad()

                outputs = self.model(images)

                loss = self.criterion(outputs, labels)

                loss.backward()

                optimizer.step()

        # Perform fine-tuning

        self.fine_tune()
 
        return self.get_parameters(config), len(self.train_loader.dataset), {}
 
    def evaluate(self, parameters, config):

        self.set_parameters(parameters)

        self.model.eval()

        correct, total = 0, 0

        loss = 0.0

        with torch.no_grad():

            for images, labels in self.test_loader:

                images, labels = images.to(DEVICE), labels.to(DEVICE)

                outputs = self.model(images)

                loss += self.criterion(outputs, labels).item()

                _, predicted = torch.max(outputs.data, 1)

                total += labels.size(0)

                correct += (predicted == labels).sum().item()

        accuracy = correct / total

        print(f"Client {self.client_id}: Accuracy = {accuracy}")

        return float(loss) / len(self.test_loader), len(self.test_loader.dataset), {"accuracy": accuracy}
 
# Load the datasets

def load_data(dataset_name):

    transform = transforms.Compose([

        transforms.Resize((32, 32)),

        transforms.ToTensor(),

        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),

    ])

    if dataset_name == "cifar10":

        train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)

        test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

    elif dataset_name == "fashion_mnist":

        transform = transforms.Compose([

            transforms.Grayscale(num_output_channels=3),

            transforms.Resize((32, 32)),

            transforms.ToTensor(),

            transforms.Normalize((0.5,), (0.5,)),

        ])

        train_dataset = FashionMNIST(root='./data', train=True, download=True, transform=transform)

        test_dataset = FashionMNIST(root='./data', train=False, download=True, transform=transform)

    elif dataset_name == "svhn":

        train_dataset = SVHN(root='./data', split='train', download=True, transform=transform)

        test_dataset = SVHN(root='./data', split='test', download=True, transform=transform)

    else:

        raise ValueError(f"Unknown dataset: {dataset_name}")
 
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    return train_loader, test_loader
 
# Updated client_fn to avoid using Context's client_id

def client_fn(cid: str) -> fl.client.Client:

    if cid == "0":

        return cifar_client

    elif cid == "1":

        return fashion_client

    elif cid == "2":

        return svhn_client
 
# Start federated learning

def start_federated_learning():

    global cifar_client, fashion_client, svhn_client

    cifar_train_loader, cifar_test_loader = load_data("cifar10")

    fashion_train_loader, fashion_test_loader = load_data("fashion_mnist")

    svhn_train_loader, svhn_test_loader = load_data("svhn")
 
    cifar_client = FLClient(CNN_Model("cifar10"), cifar_train_loader, cifar_test_loader, "CIFAR-10")

    fashion_client = FLClient(CNN_Model("fashion_mnist"), fashion_train_loader, fashion_test_loader, "Fashion-MNIST")

    svhn_client = FLClient(CNN_Model("svhn"), svhn_train_loader, svhn_test_loader, "SVHN")
 
    fl.simulation.start_simulation(

        client_fn=client_fn,

        num_clients=3,

        client_resources={"num_cpus": 2, "num_gpus": 1},

        config=fl.server.ServerConfig(num_rounds=5),  # Increased number of rounds

        strategy=fl.server.strategy.FedAvg(

            fraction_fit=1.0,

            min_fit_clients=3,

            min_available_clients=3,

        )

    )
 
if __name__ == "__main__":

    start_federated_learning()

 

Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-12-11 19:45:36,273	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'memory': 9886003200.0, 'GPU': 1.0, 'accelerator_type:P4000': 1.0, 'node:__internal_head__': 1.0, 'object_store_memory': 4943001600.0, 'CPU': 12.0, 'node:131.227.65.97': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 2, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(ClientAppActor pid=3505856) /user/HS402/as05511/.local/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implic

(ClientAppActor pid=3505856) Client Fashion-MNIST: Switched to SGD


(ClientAppActor pid=3505856) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3505856) 
(ClientAppActor pid=3505856)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3505856)             entirely in future versions of Flower.
(ClientAppActor pid=3505856)         
(ClientAppActor pid=3505856) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=3505856) Client CIFAR-10: Switched to SGD


...

(ClientAppActor pid=3505856) Client SVHN: Accuracy = 0.5675706822372465


(ClientAppActor pid=3505856) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3505856) 
(ClientAppActor pid=3505856)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3505856)             entirely in future versions of Flower.
(ClientAppActor pid=3505856)         
(ClientAppActor pid=3505856) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=3505856) Client Fashion-MNIST: Accuracy = 0.692


(ClientAppActor pid=3505856) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3505856) 
(ClientAppActor pid=3505856)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3505856)             entirely in future versions of Flower.
(ClientAppActor pid=3505856)         
(ClientAppActor pid=3505856) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 1612.60s
INFO :      	History (loss, distributed):
INFO :      		round 1: 2.285966410083734
INFO :    

(ClientAppActor pid=3505856) Client CIFAR-10: Accuracy = 0.289


In [3]:
import flwr as fl

import torch

import torch.nn as nn

import torch.optim as optim

import torch.nn.functional as F

from torch.utils.data import DataLoader

import torchvision.transforms as transforms 

from torchvision.datasets import CIFAR10, FashionMNIST, SVHN
 
# Constants

BATCH_SIZE = 64

LEARNING_RATE = 0.001

EPOCHS = 5

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SWITCH_EPOCH = 3  # The epoch at which we switch from Adam to SGD
 
# Define the model with dataset-specific layers

class CNN_Model(nn.Module):

    def __init__(self, dataset_name):

        super(CNN_Model, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)

        self.pool = nn.AvgPool2d(2, 2)

        self.fc_config = nn.Linear(64 * 8 * 8, 128)
 
        # Dataset-specific layers

        if dataset_name == "cifar10":

            self.fc_op = nn.Sequential(

                nn.Linear(128, 64),

                nn.ReLU(),

                nn.Linear(64, 10)

            )

        elif dataset_name == "fashion_mnist":

            self.fc_op = nn.Sequential(

                nn.Linear(128, 32),

                nn.ReLU(),

                nn.Linear(32, 10)

            )

        elif dataset_name == "svhn":

            self.fc_op = nn.Sequential(

                nn.Linear(128, 64),

                nn.ReLU(),

                nn.Linear(64, 10)

            )
 
    def forward(self, x):

        x = F.relu(self.conv1(x))

        x = self.pool(x)

        x = F.relu(self.conv2(x))

        x = self.pool(x)

        x = x.view(-1, 64 * 8 * 8)

        x = F.relu(self.fc_config(x))

        x = self.fc_op(x)

        return F.softmax(x, dim=1)
 
# Define the Flower client

class FLClient(fl.client.NumPyClient):

    def __init__(self, model, train_loader, test_loader, client_id):

        self.model = model.to(DEVICE)

        self.train_loader = train_loader

        self.test_loader = test_loader

        self.criterion = nn.CrossEntropyLoss()

        self.optimizer = optim.Adam(self.model.parameters(), lr=LEARNING_RATE)

        self.sgd = None

        self.switched = False

        self.client_id = client_id
 
    def switch_to_sgd(self):

        # Switch to SGD optimizer

        self.sgd = optim.SGD(self.model.parameters(), lr=self.optimizer.param_groups[0]['lr'])

        self.switched = True

        print(f"Client {self.client_id}: Switched to SGD")
 
    def step_optimizer(self, epoch):

        # Switch from Adam to SGD if the conditions are met

        if not self.switched and epoch >= SWITCH_EPOCH:

            self.switch_to_sgd()

        return self.sgd if self.switched else self.optimizer
 
    def get_parameters(self, config=None):

        params = {

            k: v.cpu().numpy()

            for k, v in self.model.state_dict().items()

            if not k.startswith("fc_op")

        }

        return list(params.values())
 
    def set_parameters(self, parameters):

        state_dict = self.model.state_dict()

        shared_keys = [k for k in state_dict.keys() if not k.startswith("fc_op")]

        for key, param in zip(shared_keys, parameters):

            state_dict[key] = torch.tensor(param)

        self.model.load_state_dict(state_dict, strict=False)
 
    def fine_tune(self):

        self.model.train()

        for epoch in range(2):  # Fine-tune for 2 local epochs

            optimizer = self.step_optimizer(epoch)

            for images, labels in self.train_loader:

                images, labels = images.to(DEVICE), labels.to(DEVICE)

                optimizer.zero_grad()

                outputs = self.model(images)

                loss = self.criterion(outputs, labels)

                loss.backward()

                optimizer.step()
 
    def fit(self, parameters, config):

        self.set_parameters(parameters)  # Set global model parameters

        self.model.train()

        for epoch in range(EPOCHS):  # Train on local data

            optimizer = self.step_optimizer(epoch)

            for images, labels in self.train_loader:

                images, labels = images.to(DEVICE), labels.to(DEVICE)

                optimizer.zero_grad()

                outputs = self.model(images)

                loss = self.criterion(outputs, labels)

                loss.backward()

                optimizer.step()

        # Perform fine-tuning

        self.fine_tune()
 
        return self.get_parameters(config), len(self.train_loader.dataset), {}
 
    def evaluate(self, parameters, config):

        self.set_parameters(parameters)

        self.model.eval()

        correct, total = 0, 0

        loss = 0.0

        with torch.no_grad():

            for images, labels in self.test_loader:

                images, labels = images.to(DEVICE), labels.to(DEVICE)

                outputs = self.model(images)

                loss += self.criterion(outputs, labels).item()

                _, predicted = torch.max(outputs.data, 1)

                total += labels.size(0)

                correct += (predicted == labels).sum().item()

        accuracy = correct / total

        print(f"Client {self.client_id}: Accuracy = {accuracy}")

        return float(loss) / len(self.test_loader), len(self.test_loader.dataset), {"accuracy": accuracy}
 
# Load the datasets

def load_data(dataset_name):

    transform = transforms.Compose([

        transforms.Resize((32, 32)),

        transforms.ToTensor(),

        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),

    ])

    if dataset_name == "cifar10":

        train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)

        test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

    elif dataset_name == "fashion_mnist":

        transform = transforms.Compose([

            transforms.Grayscale(num_output_channels=3),

            transforms.Resize((32, 32)),

            transforms.ToTensor(),

            transforms.Normalize((0.5,), (0.5,)),

        ])

        train_dataset = FashionMNIST(root='./data', train=True, download=True, transform=transform)

        test_dataset = FashionMNIST(root='./data', train=False, download=True, transform=transform)

    elif dataset_name == "svhn":

        train_dataset = SVHN(root='./data', split='train', download=True, transform=transform)

        test_dataset = SVHN(root='./data', split='test', download=True, transform=transform)

    else:

        raise ValueError(f"Unknown dataset: {dataset_name}")
 
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    return train_loader, test_loader
 
# Updated client_fn to avoid using Context's client_id

def client_fn(cid: str) -> fl.client.Client:

    if cid == "0":

        return cifar_client

    elif cid == "1":

        return fashion_client

    elif cid == "2":

        return svhn_client
 
# Start federated learning

def start_federated_learning():

    global cifar_client, fashion_client, svhn_client

    cifar_train_loader, cifar_test_loader = load_data("cifar10")

    fashion_train_loader, fashion_test_loader = load_data("fashion_mnist")

    svhn_train_loader, svhn_test_loader = load_data("svhn")
 
    cifar_client = FLClient(CNN_Model("cifar10"), cifar_train_loader, cifar_test_loader, "CIFAR-10")

    fashion_client = FLClient(CNN_Model("fashion_mnist"), fashion_train_loader, fashion_test_loader, "Fashion-MNIST")

    svhn_client = FLClient(CNN_Model("svhn"), svhn_train_loader, svhn_test_loader, "SVHN")
 
    fl.simulation.start_simulation(

        client_fn=client_fn,

        num_clients=3,

        client_resources={"num_cpus": 2, "num_gpus": 1},

        config=fl.server.ServerConfig(num_rounds=5),  # Updated to 5 rounds

        strategy=fl.server.strategy.FedAvg(

            fraction_fit=1.0,

            min_fit_clients=3,

            min_available_clients=3,

        )

    )
 
if __name__ == "__main__":

    start_federated_learning()


Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-12-11 16:46:30,829	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'object_store_memory': 5350174310.0, 'accelerator_type:P4000': 1.0, 'node:__internal_head__': 1.0, 'node:131.227.65.97': 1.0, 'CPU': 12.0, 'memory': 10700348622.0, 'GPU': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 2, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(ClientAppActor pid=3463827) /user/HS402/as05511/.local/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module impli

(ClientAppActor pid=3463827) Client CIFAR-10: Switched to SGD


(ClientAppActor pid=3463827) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3463827) 
(ClientAppActor pid=3463827)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3463827)             entirely in future versions of Flower.
(ClientAppActor pid=3463827)         
(ClientAppActor pid=3463827) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=3463827) Client Fashion-MNIST: Switched to SGD


...

(ClientAppActor pid=3463827) Client CIFAR-10: Accuracy = 0.3091


(ClientAppActor pid=3463827) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3463827) 
(ClientAppActor pid=3463827)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3463827)             entirely in future versions of Flower.
(ClientAppActor pid=3463827)         
(ClientAppActor pid=3463827) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=3463827) Client SVHN: Accuracy = 0.5221266133988937


(ClientAppActor pid=3463827) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3463827) 
(ClientAppActor pid=3463827)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3463827)             entirely in future versions of Flower.
(ClientAppActor pid=3463827)         
(ClientAppActor pid=3463827) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 1629.96s
INFO :      	History (loss, distributed):
INFO :      		round 1: 2.290308430598742
INFO :    

(ClientAppActor pid=3463827) Client Fashion-MNIST: Accuracy = 0.6906


In [7]:
import flwr as fl
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms 
from torchvision.datasets import CIFAR10, FashionMNIST, SVHN

# Constants
BATCH_SIZE = 64
LEARNING_RATE = 0.001
EPOCHS = 5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model with dataset-specific layers
class CNN_Model(nn.Module):
    def __init__(self, dataset_name):
        super(CNN_Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.AvgPool2d(2, 2)
        self.dropout = nn.Dropout(0.5) if dataset_name == "cifar10" else nn.Identity()
        self.fc_config = nn.Linear(64 * 8 * 8, 128)

        # Dataset-specific layers
        if dataset_name == "cifar10":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                self.dropout,
                nn.Linear(64, 10)
            )
        elif dataset_name == "fashion_mnist":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 32),
                nn.ReLU(),
                nn.Linear(32, 10)
            )
        elif dataset_name == "svhn":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 10)
            )

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc_config(x))
        x = self.fc_op(x)
        return F.softmax(x, dim=1)

# Define the Flower client
class FLClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, test_loader, client_id):
        self.model = model.to(DEVICE)
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=LEARNING_RATE)
        self.client_id = client_id

    def get_parameters(self, config=None):
        params = {
            k: v.cpu().numpy()
            for k, v in self.model.state_dict().items()
            if not k.startswith("fc_op")
        }
        return list(params.values())

    def set_parameters(self, parameters):
        state_dict = self.model.state_dict()
        shared_keys = [k for k in state_dict.keys() if not k.startswith("fc_op")]
        for key, param in zip(shared_keys, parameters):
            state_dict[key] = torch.tensor(param)
        self.model.load_state_dict(state_dict, strict=False)

    def fine_tune(self):
        self.model.train()
        for epoch in range(2):  # Fine-tune for 2 local epochs
            for images, labels in self.train_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

    def fit(self, parameters, config):
        self.set_parameters(parameters)  # Set global model parameters
        self.model.train()
        for epoch in range(EPOCHS):  # Train on local data
            for images, labels in self.train_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
        # Perform fine-tuning
        self.fine_tune()

        return self.get_parameters(config), len(self.train_loader.dataset), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        correct, total = 0, 0
        loss = 0.0
        with torch.no_grad():
            for images, labels in self.test_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = self.model(images)
                loss += self.criterion(outputs, labels).item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
        print(f"Client {self.client_id}: Accuracy = {accuracy}")
        return float(loss) / len(self.test_loader), len(self.test_loader.dataset), {"accuracy": accuracy}

# Load the datasets
def load_data(dataset_name):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    if dataset_name == "cifar10":
        train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
        test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
    elif dataset_name == "fashion_mnist":
        transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,)),
        ])
        train_dataset = FashionMNIST(root='./data', train=True, download=True, transform=transform)
        test_dataset = FashionMNIST(root='./data', train=False, download=True, transform=transform)
    elif dataset_name == "svhn":
        train_dataset = SVHN(root='./data', split='train', download=True, transform=transform)
        test_dataset = SVHN(root='./data', split='test', download=True, transform=transform)
    else:
        raise ValueError(f"Unknown dataset: {dataset_name}")

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    return train_loader, test_loader

# Updated client_fn to avoid using Context's client_id
def client_fn(cid: str) -> fl.client.Client:
    if cid == "0":
        return cifar_client
    elif cid == "1":
        return fashion_client
    elif cid == "2":
        return svhn_client

# Start federated learning
def start_federated_learning():
    global cifar_client, fashion_client, svhn_client
    cifar_train_loader, cifar_test_loader = load_data("cifar10")
    fashion_train_loader, fashion_test_loader = load_data("fashion_mnist")
    svhn_train_loader, svhn_test_loader = load_data("svhn")

    cifar_client = FLClient(CNN_Model("cifar10"), cifar_train_loader, cifar_test_loader, "CIFAR-10")
    fashion_client = FLClient(CNN_Model("fashion_mnist"), fashion_train_loader, fashion_test_loader, "Fashion-MNIST")
    svhn_client = FLClient(CNN_Model("svhn"), svhn_train_loader, svhn_test_loader, "SVHN")

    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=3,
        client_resources={"num_cpus": 2, "num_gpus": 1},
        config=fl.server.ServerConfig(num_rounds=10),  # Increased number of rounds
        strategy=fl.server.strategy.FedAvg(
            fraction_fit=1.0,
            min_fit_clients=3,
            min_available_clients=3,
        )
    )

if __name__ == "__main__":
    start_federated_learning()


Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout
2024-12-11 17:36:25,297	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'object_store_memory': 4941254246.0, 'memory': 9882508494.0, 'node:131.227.65.97': 1.0, 'CPU': 12.0, 'node:__internal_head__': 1.0, 'accelerator_type:P4000': 1.0, 'GPU': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 2, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(ClientAppActor pid=3477374) /user/HS402/as05511/.local/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module impli

(ClientAppActor pid=3477374) Client SVHN: Accuracy = 0.2796558082360172


(ClientAppActor pid=3477374) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3477374) 
(ClientAppActor pid=3477374)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3477374)             entirely in future versions of Flower.
(ClientAppActor pid=3477374)         
(ClientAppActor pid=3477374) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=3477374) Client CIFAR-10: Accuracy = 0.1131


...

(ClientAppActor pid=3477374) Client Fashion-MNIST: Accuracy = 0.5683


(ClientAppActor pid=3477374) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3477374) 
(ClientAppActor pid=3477374)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3477374)             entirely in future versions of Flower.
(ClientAppActor pid=3477374)         
(ClientAppActor pid=3477374) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=3477374) Client SVHN: Accuracy = 0.468000921942225


(ClientAppActor pid=3477374) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3477374) 
(ClientAppActor pid=3477374)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3477374)             entirely in future versions of Flower.
(ClientAppActor pid=3477374)         
(ClientAppActor pid=3477374) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 round(s) in 3290.23s
INFO :      	History (loss, distributed):
INFO :      		round 1: 2.2972366015501593
INFO :  

(ClientAppActor pid=3477374) Client CIFAR-10: Accuracy = 0.2598


In [ ]:
####FEDPROXXX

In [11]:
import flwr as fl
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10, FashionMNIST, SVHN

# Constants
BATCH_SIZE = 128  # Default batch size for proper processing
LEARNING_RATE = 0.001  # Default learning rate for stability
EPOCHS = 10  # Standard epochs for training
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model with dataset-specific layers
class CNN_Model(nn.Module):
    def __init__(self, dataset_name):
        super(CNN_Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.AvgPool2d(2, 2)
        self.fc_config = nn.Linear(64 * 8 * 8, 128)

        # Dataset-specific layers
        if dataset_name == "cifar10":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 10)
            )
        elif dataset_name == "fashion_mnist":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 32),
                nn.ReLU(),
                nn.Linear(32, 10)
            )
        elif dataset_name == "svhn":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 10)
            )

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc_config(x))
        x = self.fc_op(x)
        return F.softmax(x, dim=1)

# Define the Flower client with PFL using FedProx
class FLClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, test_loader, client_id):
        self.model = model.to(DEVICE)
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=LEARNING_RATE)  # Default optimizer for proper training
        self.client_id = client_id

    def get_parameters(self, config=None):
        params = {
            k: v.cpu().numpy()
            for k, v in self.model.state_dict().items()
            if not k.startswith("fc_op")
        }
        return list(params.values())

    def set_parameters(self, parameters):
        state_dict = self.model.state_dict()
        shared_keys = [k for k in state_dict.keys() if not k.startswith("fc_op")]
        for key, param in zip(shared_keys, parameters):
            state_dict[key] = torch.tensor(param)
        self.model.load_state_dict(state_dict, strict=False)

    def fit(self, parameters, config):
        # Set global parameters on the local model
        self.set_parameters(parameters)
        self.model.train()
        
        # Save global model parameters for proximity
        global_params = [param.clone().detach().to(DEVICE) for param in self.model.parameters()]
        
        for epoch in range(EPOCHS):
            for images, labels in self.train_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                
                # Add proximal term to the loss
                prox_term = 0.0
                for param, global_param in zip(self.model.parameters(), global_params):
                    prox_term += torch.norm(param - global_param) ** 2
                loss += (0.01 / 2) * prox_term  # 0.01 is the proximal coefficient
                
                loss.backward()
                self.optimizer.step()
        
        # Fine-tune locally for personalization
        for epoch in range(2):  # Fine-tune for 2 local epochs
            for images, labels in self.train_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
        
        return self.get_parameters(config), len(self.train_loader.dataset), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        correct, total = 0, 0
        loss = 0.0
        with torch.no_grad():
            for images, labels in self.test_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = self.model(images)
                loss += self.criterion(outputs, labels).item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
        print(f"Client {self.client_id}: Accuracy = {accuracy}")
        return float(loss) / len(self.test_loader), len(self.test_loader.dataset), {"accuracy": accuracy}

# Load the datasets
def load_data(dataset_name):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    if dataset_name == "cifar10":
        train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
        test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
    elif dataset_name == "fashion_mnist": 
        transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,)),
        ])
        train_dataset = FashionMNIST(root='./data', train=True, download=True, transform=transform)
        test_dataset = FashionMNIST(root='./data', train=False, download=True, transform=transform)
    elif dataset_name == "svhn":
        train_dataset = SVHN(root='./data', split='train', download=True, transform=transform)
        test_dataset = SVHN(root='./data', split='test', download=True, transform=transform)
    else:
        raise ValueError(f"Unknown dataset: {dataset_name}")

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    return train_loader, test_loader

# Updated client_fn to avoid using Context's client_id
def client_fn(cid: str) -> fl.client.Client:
    if cid == "0":
        return cifar_client
    elif cid == "1":
        return fashion_client
    elif cid == "2":
        return svhn_client

# Start federated learning
def start_federated_learning():
    global cifar_client, fashion_client, svhn_client
    cifar_train_loader, cifar_test_loader = load_data("cifar10")
    fashion_train_loader, fashion_test_loader = load_data("fashion_mnist")
    svhn_train_loader, svhn_test_loader = load_data("svhn")

    cifar_client = FLClient(CNN_Model("cifar10"), cifar_train_loader, cifar_test_loader, "CIFAR-10")
    fashion_client = FLClient(CNN_Model("fashion_mnist"), fashion_train_loader, fashion_test_loader, "Fashion-MNIST")
    svhn_client = FLClient(CNN_Model("svhn"), svhn_train_loader, svhn_test_loader, "SVHN")

    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=3,
        client_resources={"num_cpus": 4, "num_gpus": 1},
        config=fl.server.ServerConfig(num_rounds=10),  # Increased number of rounds for proper results
        strategy=fl.server.strategy.FedAvg(
            fraction_fit=1.0,
            min_fit_clients=3,
            min_available_clients=3,
        )
    )

if __name__ == "__main__":
    start_federated_learning()


Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout
2024-12-12 19:41:14,201	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'object_store_memory': 5006184038.0, 'memory': 10012368078.0, 'GPU': 1.0, 'accelerator_type:P4000': 1.0, 'node:__internal_head__': 1.0, 'node:131.227.65.97': 1.0, 'CPU': 12.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 4, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(ClientAppActor pid=3810323) /user/HS402/as05511/.local/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module impl

(ClientAppActor pid=3810323) Client CIFAR-10: Accuracy = 0.1413


(ClientAppActor pid=3810323) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3810323) 
(ClientAppActor pid=3810323)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3810323)             entirely in future versions of Flower.
(ClientAppActor pid=3810323)         
(ClientAppActor pid=3810323) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=3810323) Client Fashion-MNIST: Accuracy = 0.1808


...

(ClientAppActor pid=3810323) Client Fashion-MNIST: Accuracy = 0.5794


(ClientAppActor pid=3810323) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3810323) 
(ClientAppActor pid=3810323)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3810323)             entirely in future versions of Flower.
(ClientAppActor pid=3810323)         
(ClientAppActor pid=3810323) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=3810323) Client SVHN: Accuracy = 0.3524124154886294


(ClientAppActor pid=3810323) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3810323) 
(ClientAppActor pid=3810323)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3810323)             entirely in future versions of Flower.
(ClientAppActor pid=3810323)         
(ClientAppActor pid=3810323) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 round(s) in 5081.95s
INFO :      	History (loss, distributed):
INFO :      		round 1: 2.293990036116014
INFO :   

(ClientAppActor pid=3810323) Client CIFAR-10: Accuracy = 0.1912


In [ ]:
import flwr as fl
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10, FashionMNIST, SVHN

# Constants
BATCH_SIZE = 128  # Default batch size for proper processing
LEARNING_RATE = 0.001  # Default learning rate for stability
EPOCHS = 10  # Standard epochs for training
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model with dataset-specific layers
class CNN_Model(nn.Module):
    def __init__(self, dataset_name):
        super(CNN_Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.AvgPool2d(2, 2)
        self.fc_config = nn.Linear(64 * 8 * 8, 128)

        # Dataset-specific layers
        if dataset_name == "cifar10":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 10)
            )
        elif dataset_name == "fashion_mnist":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 32),
                nn.ReLU(),
                nn.Linear(32, 10)
            )
        elif dataset_name == "svhn":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 10)
            )

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc_config(x))
        x = self.fc_op(x)
        return F.softmax(x, dim=1)

# Define the Flower client with PFL using FedProx
class FLClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, test_loader, client_id):
        self.model = model.to(DEVICE)
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=LEARNING_RATE)  # Default optimizer for proper training
        self.client_id = client_id

    def get_parameters(self, config=None):
        params = {
            k: v.cpu().numpy()
            for k, v in self.model.state_dict().items()
            if not k.startswith("fc_op")
        }
        return list(params.values())

    def set_parameters(self, parameters):
        state_dict = self.model.state_dict()
        shared_keys = [k for k in state_dict.keys() if not k.startswith("fc_op")]
        for key, param in zip(shared_keys, parameters):
            state_dict[key] = torch.tensor(param)
        self.model.load_state_dict(state_dict, strict=False)

    def fit(self, parameters, config):
        # Set global parameters on the local model
        self.set_parameters(parameters)
        self.model.train()
        
        # Save global model parameters for proximity
        global_params = [param.clone().detach().to(DEVICE) for param in self.model.parameters()]
        
        for epoch in range(EPOCHS):
            for images, labels in self.train_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                
                # Add proximal term to the loss
                prox_term = 0.0
                for param, global_param in zip(self.model.parameters(), global_params):
                    prox_term += torch.norm(param - global_param) ** 2
                loss += (0.01 / 2) * prox_term  # 0.01 is the proximal coefficient
                
                loss.backward()
                self.optimizer.step()
        
        # Fine-tune locally for personalization
        for epoch in range(2):  # Fine-tune for 2 local epochs
            for images, labels in self.train_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
        
        return self.get_parameters(config), len(self.train_loader.dataset), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        correct, total = 0, 0
        loss = 0.0
        with torch.no_grad():
            for images, labels in self.test_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = self.model(images)
                loss += self.criterion(outputs, labels).item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
        print(f"Client {self.client_id}: Accuracy = {accuracy}")
        return float(loss) / len(self.test_loader), len(self.test_loader.dataset), {"accuracy": accuracy}

# Load the datasets
def load_data(dataset_name):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    if dataset_name == "cifar10":
        train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
        test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
    elif dataset_name == "fashion_mnist": 
        transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,)),
        ])
        train_dataset = FashionMNIST(root='./data', train=True, download=True, transform=transform)
        test_dataset = FashionMNIST(root='./data', train=False, download=True, transform=transform)
    elif dataset_name == "svhn":
        train_dataset = SVHN(root='./data', split='train', download=True, transform=transform)
        test_dataset = SVHN(root='./data', split='test', download=True, transform=transform)
    else:
        raise ValueError(f"Unknown dataset: {dataset_name}")

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    return train_loader, test_loader

# Updated client_fn to avoid using Context's client_id
def client_fn(cid: str) -> fl.client.Client:
    if cid == "0":
        return cifar_client
    elif cid == "1":
        return fashion_client
    elif cid == "2":
        return svhn_client

# Start federated learning
def start_federated_learning():
    global cifar_client, fashion_client, svhn_client
    cifar_train_loader, cifar_test_loader = load_data("cifar10")
    fashion_train_loader, fashion_test_loader = load_data("fashion_mnist")
    svhn_train_loader, svhn_test_loader = load_data("svhn")

    cifar_client = FLClient(CNN_Model("cifar10"), cifar_train_loader, cifar_test_loader, "CIFAR-10")
    fashion_client = FLClient(CNN_Model("fashion_mnist"), fashion_train_loader, fashion_test_loader, "Fashion-MNIST")
    svhn_client = FLClient(CNN_Model("svhn"), svhn_train_loader, svhn_test_loader, "SVHN")

    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=3,
        client_resources={"num_cpus": 4, "num_gpus": 1},
        config=fl.server.ServerConfig(num_rounds=20),  # Increased number of rounds for proper results
        strategy=fl.server.strategy.FedAvg(
            fraction_fit=1.0,
            min_fit_clients=3,
            min_available_clients=3,
        )
    )

if __name__ == "__main__":
    start_federated_learning()


2025-01-03 21:22:44,790	INFO util.py:154 -- Outdated packages:
  ipywidgets==6.0.0 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:
import flwr as fl
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10, FashionMNIST, SVHN

# Constants
BATCH_SIZE = 64  # Default batch size for proper processing
LEARNING_RATE = 0.001  # Default learning rate for stability
EPOCHS = 5  # Standard epochs for training
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model with dataset-specific layers
class CNN_Model(nn.Module):
    def __init__(self, dataset_name):
        super(CNN_Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.AvgPool2d(2, 2)
        self.fc_config = nn.Linear(64 * 8 * 8, 128)

        # Dataset-specific layers
        if dataset_name == "cifar10":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 10)
            )
        elif dataset_name == "fashion_mnist":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 32),
                nn.ReLU(),
                nn.Linear(32, 10)
            )
        elif dataset_name == "svhn":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 10)
            )

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc_config(x))
        x = self.fc_op(x)
        return F.softmax(x, dim=1)

# Define the Flower client with PFL using FedProx
class FLClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, test_loader, client_id):
        self.model = model.to(DEVICE)
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=LEARNING_RATE)  # Default optimizer for proper training
        self.client_id = client_id

    def get_parameters(self, config=None):
        params = {
            k: v.cpu().numpy()
            for k, v in self.model.state_dict().items()
            if not k.startswith("fc_op")
        }
        return list(params.values())

    def set_parameters(self, parameters):
        state_dict = self.model.state_dict()
        shared_keys = [k for k in state_dict.keys() if not k.startswith("fc_op")]
        for key, param in zip(shared_keys, parameters):
            state_dict[key] = torch.tensor(param)
        self.model.load_state_dict(state_dict, strict=False)

    def fit(self, parameters, config):
        # Set global parameters on the local model
        self.set_parameters(parameters)
        self.model.train()
        
        # Save global model parameters for proximity
        global_params = [param.clone().detach().to(DEVICE) for param in self.model.parameters()]
        
        for epoch in range(EPOCHS):
            for images, labels in self.train_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                
                # Add proximal term to the loss
                prox_term = 0.0
                for param, global_param in zip(self.model.parameters(), global_params):
                    prox_term += torch.norm(param - global_param) ** 2
                loss += (0.01 / 2) * prox_term  # 0.01 is the proximal coefficient
                
                loss.backward()
                self.optimizer.step()
        
        # Fine-tune locally for personalization
        for epoch in range(2):  # Fine-tune for 2 local epochs
            for images, labels in self.train_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
        
        return self.get_parameters(config), len(self.train_loader.dataset), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        correct, total = 0, 0
        loss = 0.0
        with torch.no_grad():
            for images, labels in self.test_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = self.model(images)
                loss += self.criterion(outputs, labels).item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
        print(f"Client {self.client_id}: Accuracy = {accuracy}")
        return float(loss) / len(self.test_loader), len(self.test_loader.dataset), {"accuracy": accuracy}

# Load the datasets
def load_data(dataset_name):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    if dataset_name == "cifar10":
        train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
        test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
    elif dataset_name == "fashion_mnist": 
        transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,)),
        ])
        train_dataset = FashionMNIST(root='./data', train=True, download=True, transform=transform)
        test_dataset = FashionMNIST(root='./data', train=False, download=True, transform=transform)
    elif dataset_name == "svhn":
        train_dataset = SVHN(root='./data', split='train', download=True, transform=transform)
        test_dataset = SVHN(root='./data', split='test', download=True, transform=transform)
    else:
        raise ValueError(f"Unknown dataset: {dataset_name}")

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    return train_loader, test_loader

# Updated client_fn to avoid using Context's client_id
def client_fn(cid: str) -> fl.client.Client:
    if cid == "0":
        return cifar_client
    elif cid == "1":
        return fashion_client
    elif cid == "2":
        return svhn_client

# Start federated learning
def start_federated_learning():
    global cifar_client, fashion_client, svhn_client
    cifar_train_loader, cifar_test_loader = load_data("cifar10")
    fashion_train_loader, fashion_test_loader = load_data("fashion_mnist")
    svhn_train_loader, svhn_test_loader = load_data("svhn")

    cifar_client = FLClient(CNN_Model("cifar10"), cifar_train_loader, cifar_test_loader, "CIFAR-10")
    fashion_client = FLClient(CNN_Model("fashion_mnist"), fashion_train_loader, fashion_test_loader, "Fashion-MNIST")
    svhn_client = FLClient(CNN_Model("svhn"), svhn_train_loader, svhn_test_loader, "SVHN")

    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=3,
        client_resources={"num_cpus": 4, "num_gpus": 1},
        config=fl.server.ServerConfig(num_rounds=5),  # Increased number of rounds for proper results
        strategy=fl.server.strategy.FedAvg(
            fraction_fit=1.0,
            min_fit_clients=3,
            min_available_clients=3,
        )
    )

if __name__ == "__main__":
    start_federated_learning()


In [ ]:
##ADaptive lambda 

In [12]:
import flwr as fl
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10, FashionMNIST, SVHN
 
# Constants
BATCH_SIZE = 32  # Default batch size for proper processing
LEARNING_RATE = 0.001  # Default learning rate for stability
EPOCHS = 2  # Standard epochs for training
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
# Define the model with dataset-specific layers
class CNN_Model(nn.Module):
    def __init__(self, dataset_name):
        super(CNN_Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.AvgPool2d(2, 2)
        self.fc_config = nn.Linear(64 * 8 * 8, 128)
 
        # Dataset-specific layers
        if dataset_name == "cifar10":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 10)
            )
        elif dataset_name == "fashion_mnist":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 32),
                nn.ReLU(),
                nn.Linear(32, 10)
            )
        elif dataset_name == "svhn":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 10)
            )
 
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc_config(x))
        x = self.fc_op(x)
        return F.softmax(x, dim=1)
 
# Helper functions for heterogeneity and dynamic lambda
def compute_statistical_heterogeneity(global_params, local_params):
    heterogeneity = 0.0
    for global_param, local_param in zip(global_params, local_params):
        heterogeneity += torch.norm(local_param - global_param) ** 2
    return heterogeneity
 
def update_lambda(heterogeneity, alpha=0.1, epsilon=1e-5):
    return alpha / (heterogeneity + epsilon)
 
# Define the Flower client with PFL using FedProx
class FLClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, test_loader, client_id):
        self.model = model.to(DEVICE)
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=LEARNING_RATE)
        self.client_id = client_id
 
    def get_parameters(self, config=None):
        params = {
            k: v.cpu().numpy()
            for k, v in self.model.state_dict().items()
            if not k.startswith("fc_op")
        }
        return list(params.values())
 
    def set_parameters(self, parameters):
        state_dict = self.model.state_dict()
        shared_keys = [k for k in state_dict.keys() if not k.startswith("fc_op")]
        for key, param in zip(shared_keys, parameters):
            state_dict[key] = torch.tensor(param)
        self.model.load_state_dict(state_dict, strict=False)
 
    def fit(self, parameters, config):
        # Set global parameters on the local model
        self.set_parameters(parameters)
        self.model.train()
 
        # Save global model parameters for proximity
        global_params = [param.clone().detach().to(DEVICE) for param in self.model.parameters()]
        for epoch in range(EPOCHS):
            for images, labels in self.train_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
 
                # Calculate heterogeneity and update lambda
                heterogeneity = compute_statistical_heterogeneity(global_params, self.model.parameters())
                lambda_val = update_lambda(heterogeneity)
 
                # Add proximal term with dynamic lambda
                prox_term = 0.0
                for param, global_param in zip(self.model.parameters(), global_params):
                    prox_term += torch.norm(param - global_param) ** 2
                loss += (lambda_val / 2) * prox_term
 
                loss.backward()
                self.optimizer.step()
 
        # Fine-tune locally for personalization
        for epoch in range(2):  # Fine-tune for 2 local epochs
            for images, labels in self.train_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
        return self.get_parameters(config), len(self.train_loader.dataset), {}
 
    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        correct, total = 0, 0
        loss = 0.0
        with torch.no_grad():
            for images, labels in self.test_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = self.model(images)
                loss += self.criterion(outputs, labels).item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
        print(f"Client {self.client_id}: Accuracy = {accuracy}")
        return float(loss) / len(self.test_loader), len(self.test_loader.dataset), {"accuracy": accuracy}
 
# Load the datasets
def load_data(dataset_name):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    if dataset_name == "cifar10":
        train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
        test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
    elif dataset_name == "fashion_mnist": 
        transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,)),
        ])
        train_dataset = FashionMNIST(root='./data', train=True, download=True, transform=transform)
        test_dataset = FashionMNIST(root='./data', train=False, download=True, transform=transform)
    elif dataset_name == "svhn":
        train_dataset = SVHN(root='./data', split='train', download=True, transform=transform)
        test_dataset = SVHN(root='./data', split='test', download=True, transform=transform)
    else:
        raise ValueError(f"Unknown dataset: {dataset_name}")
 
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    return train_loader, test_loader
 
# Updated client_fn to avoid using Context's client_id
def client_fn(cid: str) -> fl.client.Client:
    if cid == "0":
        return cifar_client
    elif cid == "1":
        return fashion_client
    elif cid == "2":
        return svhn_client
 
# Start federated learning
def start_federated_learning():
    global cifar_client, fashion_client, svhn_client
    cifar_train_loader, cifar_test_loader = load_data("cifar10")
    fashion_train_loader, fashion_test_loader = load_data("fashion_mnist")
    svhn_train_loader, svhn_test_loader = load_data("svhn")
 
    cifar_client = FLClient(CNN_Model("cifar10"), cifar_train_loader, cifar_test_loader, "CIFAR-10")
    fashion_client = FLClient(CNN_Model("fashion_mnist"), fashion_train_loader, fashion_test_loader, "Fashion-MNIST")
    svhn_client = FLClient(CNN_Model("svhn"), svhn_train_loader, svhn_test_loader, "SVHN")
 
    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=3,
        client_resources={"num_cpus": 4, "num_gpus": 1},
        config=fl.server.ServerConfig(num_rounds=3),
        strategy=fl.server.strategy.FedAvg(
            fraction_fit=1.0,
            min_fit_clients=3,
            min_available_clients=3,
        )
    )
 
if __name__ == "__main__":
    start_federated_learning()

Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


INFO :      Starting Flower simulation, config: num_rounds=3, no round_timeout
2024-12-12 21:08:56,137	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'GPU': 1.0, 'CPU': 12.0, 'node:131.227.65.97': 1.0, 'memory': 10003867239.0, 'node:__internal_head__': 1.0, 'accelerator_type:P4000': 1.0, 'object_store_memory': 5001933619.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 4, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(ClientAppActor pid=3827597) /user/HS402/as05511/.local/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module impli

(ClientAppActor pid=3827597) Client SVHN: Accuracy = 0.25069145666871545


(ClientAppActor pid=3827597) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3827597) 
(ClientAppActor pid=3827597)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3827597)             entirely in future versions of Flower.
(ClientAppActor pid=3827597)         
(ClientAppActor pid=3827597) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=3827597) Client CIFAR-10: Accuracy = 0.1723


...

(ClientAppActor pid=3827597) Client Fashion-MNIST: Accuracy = 0.5361


(ClientAppActor pid=3827597) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3827597) 
(ClientAppActor pid=3827597)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3827597)             entirely in future versions of Flower.
(ClientAppActor pid=3827597)         
(ClientAppActor pid=3827597) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=3827597) Client SVHN: Accuracy = 0.3898279041180086


(ClientAppActor pid=3827597) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3827597) 
(ClientAppActor pid=3827597)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3827597)             entirely in future versions of Flower.
(ClientAppActor pid=3827597)         
(ClientAppActor pid=3827597) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 747.66s
INFO :      	History (loss, distributed):
INFO :      		round 1: 2.296437486170988
INFO :     

(ClientAppActor pid=3827597) Client CIFAR-10: Accuracy = 0.204


In [ ]:
import flwr as fl
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10, FashionMNIST, SVHN
 
# Constants
BATCH_SIZE = 128  # Default batch size for proper processing
LEARNING_RATE = 0.001  # Default learning rate for stability
EPOCHS = 10  # Standard epochs for training
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
# Define the model with dataset-specific layers
class CNN_Model(nn.Module):
    def __init__(self, dataset_name):
        super(CNN_Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.AvgPool2d(2, 2)
        self.fc_config = nn.Linear(64 * 8 * 8, 128)
 
        # Dataset-specific layers
        if dataset_name == "cifar10":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 10)
            )
        elif dataset_name == "fashion_mnist":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 32),
                nn.ReLU(),
                nn.Linear(32, 10)
            )
        elif dataset_name == "svhn":
            self.fc_op = nn.Sequential(
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 10)
            )
 
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc_config(x))
        x = self.fc_op(x)
        return F.softmax(x, dim=1)
 
# Helper functions for heterogeneity and dynamic lambda
def compute_statistical_heterogeneity(global_params, local_params):
    heterogeneity = 0.0
    for global_param, local_param in zip(global_params, local_params):
        heterogeneity += torch.norm(local_param - global_param) ** 2
    return heterogeneity
 
def update_lambda(heterogeneity, alpha=0.1, epsilon=1e-5):
    return alpha / (heterogeneity + epsilon)
 
# Define the Flower client with PFL using FedProx
class FLClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, test_loader, client_id):
        self.model = model.to(DEVICE)
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=LEARNING_RATE)
        self.client_id = client_id
 
    def get_parameters(self, config=None):
        params = {
            k: v.cpu().numpy()
            for k, v in self.model.state_dict().items()
            if not k.startswith("fc_op")
        }
        return list(params.values())
 
    def set_parameters(self, parameters):
        state_dict = self.model.state_dict()
        shared_keys = [k for k in state_dict.keys() if not k.startswith("fc_op")]
        for key, param in zip(shared_keys, parameters):
            state_dict[key] = torch.tensor(param)
        self.model.load_state_dict(state_dict, strict=False)
 
    def fit(self, parameters, config):
        # Set global parameters on the local model
        self.set_parameters(parameters)
        self.model.train()
 
        # Save global model parameters for proximity
        global_params = [param.clone().detach().to(DEVICE) for param in self.model.parameters()]
        for epoch in range(EPOCHS):
            for images, labels in self.train_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
 
                # Calculate heterogeneity and update lambda
                heterogeneity = compute_statistical_heterogeneity(global_params, self.model.parameters())
                lambda_val = update_lambda(heterogeneity)
 
                # Add proximal term with dynamic lambda
                prox_term = 0.0
                for param, global_param in zip(self.model.parameters(), global_params):
                    prox_term += torch.norm(param - global_param) ** 2
                loss += (lambda_val / 2) * prox_term
 
                loss.backward()
                self.optimizer.step()
 
        # Fine-tune locally for personalization
        for epoch in range(2):  # Fine-tune for 2 local epochs
            for images, labels in self.train_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
        return self.get_parameters(config), len(self.train_loader.dataset), {}
 
    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        correct, total = 0, 0
        loss = 0.0
        with torch.no_grad():
            for images, labels in self.test_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = self.model(images)
                loss += self.criterion(outputs, labels).item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
        print(f"Client {self.client_id}: Accuracy = {accuracy}")
        return float(loss) / len(self.test_loader), len(self.test_loader.dataset), {"accuracy": accuracy}
 
# Load the datasets
def load_data(dataset_name):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    if dataset_name == "cifar10":
        train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
        test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
    elif dataset_name == "fashion_mnist": 
        transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,)),
        ])
        train_dataset = FashionMNIST(root='./data', train=True, download=True, transform=transform)
        test_dataset = FashionMNIST(root='./data', train=False, download=True, transform=transform)
    elif dataset_name == "svhn":
        train_dataset = SVHN(root='./data', split='train', download=True, transform=transform)
        test_dataset = SVHN(root='./data', split='test', download=True, transform=transform)
    else:
        raise ValueError(f"Unknown dataset: {dataset_name}")
 
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    return train_loader, test_loader
 
# Updated client_fn to avoid using Context's client_id
def client_fn(cid: str) -> fl.client.Client:
    if cid == "0":
        return cifar_client
    elif cid == "1":
        return fashion_client
    elif cid == "2":
        return svhn_client
 
# Start federated learning
def start_federated_learning():
    global cifar_client, fashion_client, svhn_client
    cifar_train_loader, cifar_test_loader = load_data("cifar10")
    fashion_train_loader, fashion_test_loader = load_data("fashion_mnist")
    svhn_train_loader, svhn_test_loader = load_data("svhn")
 
    cifar_client = FLClient(CNN_Model("cifar10"), cifar_train_loader, cifar_test_loader, "CIFAR-10")
    fashion_client = FLClient(CNN_Model("fashion_mnist"), fashion_train_loader, fashion_test_loader, "Fashion-MNIST")
    svhn_client = FLClient(CNN_Model("svhn"), svhn_train_loader, svhn_test_loader, "SVHN")
 
    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=3,
        client_resources={"num_cpus": 4, "num_gpus": 1},
        config=fl.server.ServerConfig(num_rounds=10),
        strategy=fl.server.strategy.FedAvg(
            fraction_fit=1.0,
            min_fit_clients=3,
            min_available_clients=3,
        )
    )
 
if __name__ == "__main__":
    start_federated_learning()

Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


INFO :      Starting Flower simulation, config: num_rounds=10, no round_timeout
2024-12-12 21:35:24,797	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'GPU': 1.0, 'object_store_memory': 5134589952.0, 'memory': 10269179904.0, 'accelerator_type:P4000': 1.0, 'node:__internal_head__': 1.0, 'CPU': 12.0, 'node:131.227.65.97': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 4, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(ClientAppActor pid=3832905) /user/HS402/as05511/.local/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module impl

(ClientAppActor pid=3832905) Client Fashion-MNIST: Accuracy = 0.2978


(ClientAppActor pid=3832905) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3832905) 
(ClientAppActor pid=3832905)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3832905)             entirely in future versions of Flower.
(ClientAppActor pid=3832905)         
(ClientAppActor pid=3832905) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=3832905) Client CIFAR-10: Accuracy = 0.1515


...

(ClientAppActor pid=3832905) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3832905) 
(ClientAppActor pid=3832905)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3832905)             entirely in future versions of Flower.
(ClientAppActor pid=3832905)         
(ClientAppActor pid=3832905) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=3832905) Client Fashion-MNIST: Accuracy = 0.5811


(ClientAppActor pid=3832905) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3832905) 
(ClientAppActor pid=3832905)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3832905)             entirely in future versions of Flower.
(ClientAppActor pid=3832905)         
(ClientAppActor pid=3832905) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 3 clients (out of 3)


(ClientAppActor pid=3832905) Client CIFAR-10: Accuracy = 0.2003


(ClientAppActor pid=3832905) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=3832905) 
(ClientAppActor pid=3832905)             This is a deprecated feature. It will be removed
(ClientAppActor pid=3832905)             entirely in future versions of Flower.
(ClientAppActor pid=3832905)         
(ClientAppActor pid=3832905) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
(ClientAppActor pid=3832905) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this